In [3]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn

import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torchvision.models import resnet50
import matplotlib.pyplot as plt
import time
import os
import copy

cudnn.benchmark = True
plt.ion()

In [5]:
#laod data
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

data_dir = './data_2_categories'
train_datesets = datasets.ImageFolder(os.path.join(data_dir, 'train'), data_transforms['train'])
train_loaders = torch.utils.data.DataLoader(train_datesets, batch_size=32, shuffle=True, num_workers=8)
train_dataset_sizes = len(train_datesets)

val_datesets = datasets.ImageFolder(os.path.join(data_dir, 'val'), data_transforms['val'])
val_loaders = torch.utils.data.DataLoader(val_datesets, batch_size=32, shuffle=True, num_workers=8)
val_dataset_sizes = len(val_datesets)

test_datesets = datasets.ImageFolder(os.path.join(data_dir, 'test'), data_transforms['test'])
test_loaders = torch.utils.data.DataLoader(test_datesets, batch_size=32, shuffle=True, num_workers=8)
test_dataset_sizes = len(test_datesets)

class_name = train_datesets.classes
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [6]:
model_tuned = models.resnet50(weights="IMAGENET1K_V1")

num_features = model_tuned.fc.in_features
output_size = 2
model_tuned.fc = nn.Linear(num_features, output_size)
model_tuned = model_tuned.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_tuned.parameters(), lr=1e-3)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /home/annika/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100.0%


In [7]:
max_iters = 25
train_losses = []
train_accs = []
for epoch in range(max_iters):
    running_loss = 0
    avg_acc = 0
    
    for i, data in enumerate(train_loaders):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model_tuned(inputs)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()

        avg_acc += ((torch.argmax(outputs,axis=1) == labels).sum() / 32).item()
        running_loss += loss.item()

    avg_loss = running_loss/ len(train_loaders)
    avg_acc /= len(train_loaders)
    train_accs.append(avg_acc)
    train_losses.append(avg_loss)

    if epoch % 1 == 0:
        print("itr: {:02d} \t loss: {:.2f} \t acc : {:.2f}".format(epoch,avg_loss,avg_acc))

itr: 00 	 loss: 0.46 	 acc : 0.78
itr: 01 	 loss: 0.33 	 acc : 0.84
itr: 02 	 loss: 0.30 	 acc : 0.86
itr: 03 	 loss: 0.30 	 acc : 0.86
itr: 04 	 loss: 0.28 	 acc : 0.87
itr: 05 	 loss: 0.24 	 acc : 0.89
itr: 06 	 loss: 0.25 	 acc : 0.88
itr: 07 	 loss: 0.22 	 acc : 0.90


KeyboardInterrupt: 